In [1]:
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.dataset as ds
import rasterio
from rasterio.windows import Window
import vaex
import numpy as np
import gc

In [2]:
print(pa.__version__)

7.0.0


In [ ]:
ds = ds.dataset("sample_dataset", format="parquet")

In [ ]:
p = ds.to_table()

In [3]:
x_off, y_off = 8000, 8000

In [4]:
mdt = rasterio.open('/media/feromes/FEROMES/MDT_sampa-ZSTD.tif')

In [5]:
window = rasterio.windows.from_bounds(331242.0, 7389124.0, 331242. + x_off/2, 7389124.0 + y_off/2, transform=mdt.transform)

In [6]:
mdt_dataset = mdt.read(1, window=window)

In [7]:
mdt_dataset.shape

(8000, 8000)

In [8]:
bhm = rasterio.open('/media/feromes/FEROMES/BHM/0-BHM-sao-paulo-city.tif')

In [9]:
bhm_dataset = bhm.read(1, window=window)

In [10]:
bhm_dataset.shape

(8000, 8000)

In [11]:
mdt_dataset

array([[814.1647 , 814.1461 , 814.1408 , ..., 726.4734 , 726.48505,
        726.4966 ],
       [814.16846, 814.1529 , 814.1526 , ..., 726.48486, 726.49567,
        726.50604],
       [814.1693 , 814.15576, 814.15955, ..., 726.4981 , 726.5075 ,
        726.5161 ],
       ...,
       [751.9352 , 752.0173 , 752.08356, ..., 790.6649 , 790.68054,
        790.69446],
       [752.0068 , 752.0829 , 752.14294, ..., 790.66376, 790.6782 ,
        790.69086],
       [752.07446, 752.1454 , 752.1998 , ..., 790.6666 , 790.6798 ,
        790.69086]], dtype=float32)

In [12]:
mds_dataset = bhm_dataset + mdt_dataset

In [13]:
del mdt_dataset
del bhm_dataset
gc.collect()

744

In [14]:
gc.collect()

0

In [15]:
mds_indices = np.array([np.where(mds_dataset > 0.)[0], np.where(mds_dataset > 0.)[1]]).transpose()

In [16]:
t = bhm.transform
mds_xy = mds_indices * (t[0], t[4]) + (t[2], t[5]) + (t[0]/2, t[4]/2)

In [17]:
mds_z = mds_dataset[mds_indices[:, 0], mds_indices[:, 1]]
mds_z

array([814.1647 , 814.1461 , 814.1408 , ..., 855.16187, 855.17505,
       855.27295], dtype=float32)

In [18]:
del mds_indices
gc.collect()

0

In [19]:
mds_xy

array([[ 313487.34, 7416122.63],
       [ 313487.34, 7416122.13],
       [ 313487.34, 7416121.63],
       ...,
       [ 317486.84, 7412124.13],
       [ 317486.84, 7412123.63],
       [ 317486.84, 7412123.13]])

In [ ]:
ds.write_dataset(p, "sample_dataset", format="parquet")

In [ ]:
p.shape

In [20]:
pa.array(mds_z).cast('double')

[
  814.1646728515625,
  814.1461181640625,
  814.1408081054688,
  814.1514282226562,
  814.1804809570312,
  814.2273559570312,
  814.2882690429688,
  814.3591918945312,
  814.4362182617188,
  814.5153198242188,
  ...
  797.6015625,
  797.765625,
  797.9115600585938,
  800.1697387695312,
  800.1930541992188,
  798.2294921875,
  798.3817749023438,
  855.161865234375,
  855.175048828125,
  855.27294921875
]

In [21]:
pc.round_to_multiple(pa.array(mds_xy[:, 0]), multiple=0.5)

[
  313487.5,
  313487.5,
  313487.5,
  313487.5,
  313487.5,
  313487.5,
  313487.5,
  313487.5,
  313487.5,
  313487.5,
  ...
  317487,
  317487,
  317487,
  317487,
  317487,
  317487,
  317487,
  317487,
  317487,
  317487
]

In [22]:
p = pa.table([pc.round_to_multiple(pa.array(mds_xy[:, 0]), multiple=0.5), 
             pc.round_to_multiple(pa.array(mds_xy[:, 1]), multiple=0.5), 
             pa.array(mds_z).cast('double')], names=['x', 'y', 'z'])

In [23]:
p

pyarrow.Table
x: double
y: double
z: double
----
x: [[313487.5,313487.5,313487.5,313487.5,313487.5,313487.5,313487.5,313487.5,313487.5,313487.5,...,317487,317487,317487,317487,317487,317487,317487,317487,317487,317487]]
y: [[7416122.5,7416122,7416121.5,7416121,7416120.5,7416120,7416119.5,7416119,7416118.5,7416118,...,7412127.5,7412127,7412126.5,7412126,7412125.5,7412125,7412124.5,7412124,7412123.5,7412123]]
z: [[814.1646728515625,814.1461181640625,814.1408081054688,814.1514282226562,814.1804809570312,814.2273559570312,814.2882690429688,814.3591918945312,814.4362182617188,814.5153198242188,...,797.6015625,797.765625,797.9115600585938,800.1697387695312,800.1930541992188,798.2294921875,798.3817749023438,855.161865234375,855.175048828125,855.27294921875]]

In [24]:
# DEve-se Considerar o resultado final em Bits
# 8 bits = 256 nuances
# 10 bits = 1024 nuances
# representacoes binarias com raiz quadradas inteiras e como 16 e 32
# Podem gerar uma divisao homogenea da semi esfera
# Por exemplo 256 resultaria em 32 fusos e 8 meridianos
# Por exemplo 1024 resultaria em 64 fusos e 16 meridianos

fusos = 32
meridianos = 8
altura_observador = 1.20

In [25]:
thetas, delta_theta = np.linspace(np.pi/2, -np.pi/2, fusos, retstep=True, endpoint=False)
thetas

array([ 1.57079633,  1.47262156,  1.37444679,  1.27627202,  1.17809725,
        1.07992247,  0.9817477 ,  0.88357293,  0.78539816,  0.68722339,
        0.58904862,  0.49087385,  0.39269908,  0.29452431,  0.19634954,
        0.09817477,  0.        , -0.09817477, -0.19634954, -0.29452431,
       -0.39269908, -0.49087385, -0.58904862, -0.68722339, -0.78539816,
       -0.88357293, -0.9817477 , -1.07992247, -1.17809725, -1.27627202,
       -1.37444679, -1.47262156])

In [26]:
phis, delta_phi = np.linspace(1., 0., meridianos, retstep=True, endpoint=False)
phis = np.arccos(phis + delta_phi/2)
phis

array([0.3554212 , 0.62236849, 0.81275556, 0.97338991, 1.11797973,
       1.25297262, 1.38217994, 1.50825556])

In [27]:
np.tan(phis)

array([ 0.37118429,  0.7174907 ,  1.05626819,  1.46986184,  2.05535637,
        3.03973683,  5.23874455, 15.96871942])

In [28]:
p = p.append_column('z_observador', pc.add(p['z'], altura_observador))

In [ ]:
%time

# x_min, y_min = 

for i_t, th in enumerate(thetas):
    
#     grid = np.zeros((x_off, y_off), dtype='uint8')

    grid = np.zeros((x_off, y_off), dtype='float32')
        
    
    for i_p, ph in enumerate(phis):
        print(f'{i_t}, {i_p} ..')
        
        d = p.append_column(f'x{i_t}_{i_p}',
                        pc.round_to_multiple(pc.add(pc.multiply(np.cos(th) * np.tan(ph), p['z']), p['x']), 
                                             multiple=0.5))
        d = d.append_column(f'y{i_t}_{i_p}',
                        pc.round_to_multiple(pc.add(pc.multiply(np.sin(th) * np.tan(ph), p['z']), p['y']), 
                                             multiple=0.5))
        
        
        g = d.group_by([f'x{i_t}_{i_p}', f'y{i_t}_{i_p}']).aggregate([("z", "max")])
        
        g = g.append_column(f'x',
                            pc.multiply(pc.subtract(pc.round_to_multiple(pc.subtract(g[f'x{i_t}_{i_p}'], 
                                       pc.multiply(g['z_max'], (np.cos(th) * np.tan(ph)))), 
                                                 multiple=0.5), pc.min(p['x'])), 2).cast('int32'))
        g = g.append_column(f'y',
                            pc.multiply(pc.subtract(pc.round_to_multiple(pc.subtract(g[f'y{i_t}_{i_p}'], 
                                       pc.multiply(g['z_max'], (np.sin(th) * np.tan(ph)))), 
                                                 multiple=0.5), pc.min(p['y'])), 2).cast('int32'))
        
        grid[g['x'],g['y']] = g['z_max']
        
#                         pc.round_to_multiple(pc.add(pc.multiply(np.cos(th) * np.tan(ph), p['z']), p['x']), 
#                                              multiple=0.5))
        
        ## É Possível calcular o x, y

#         pd_seila = seila.to_pandas(split_blocks=True, self_destruct=True)
#         pd_d = d.to_pandas(split_blocks=True, self_destruct=True)
#         joined = pd_d.join(pd_seila, how='left', rsuffix='r')
    
        
#         break
    
#     break
    

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs
0, 0 ..
0, 1 ..
0, 2 ..
0, 3 ..
0, 4 ..
0, 5 ..
0, 6 ..
0, 7 ..
1, 0 ..
1, 1 ..
1, 2 ..
1, 3 ..
1, 4 ..
1, 5 ..
1, 6 ..
1, 7 ..
2, 0 ..
2, 1 ..


In [ ]:
grid[g['x'],g['y']] = g['z_max']